In [1]:
from dnallm import load_config, load_model_and_tokenizer, DNADataset, DNATrainer

In [2]:
# Load the config file
configs = load_config("./ner_task_config.yaml")

In [3]:
# Load the model and tokenizer
model_name = "zhangtaolab/plant-nucleotide-transformer-BPE"
# from Hugging Face
# model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'], source="huggingface")
# from ModelScope
model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'], source="modelscope")

2025-09-12 23:33:35,943 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

2025-09-12 23:34:54,585 - modelscope - INFO - Download model 'zhangtaolab/plant-nucleotide-transformer-BPE' successfully.


Model files are stored in /home/liuguanqing/.cache/modelscope/hub/models/zhangtaolab/plant-nucleotide-transformer-BPE


Some weights of EsmForTokenClassification were not initialized from the model checkpoint at /home/liuguanqing/.cache/modelscope/hub/models/zhangtaolab/plant-nucleotide-transformer-BPE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Load the datasets
datasets = DNADataset.load_local_data("./rice_gene_ner_BPE.pkl", seq_col="sequence", label_col="labels", tokenizer=tokenizer, max_length=1024)

# Encode the sequences with given task's data collator
datasets.encode_sequences(task=configs['task'].task_type, remove_unused_columns=True)

# Split the dataset into train, test, and validation sets
datasets.split_data()

Format labels:   0%|          | 0/2000 [00:00<?, ? examples/s]

Encoding inputs:   0%|          | 0/2000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2067 > 2048). Running this sequence through the model will result in indexing errors


In [5]:
# Initialize the trainer
trainer = DNATrainer(
    model=model,
    config=configs,
    datasets=datasets
)

In [6]:
# Start training
metrics = trainer.train()
print(metrics)

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.571600,0.311933,0.903697,0.693850,0.631387,0.661146
1000,0.154700,0.220429,0.935598,0.775353,0.734793,0.754528


{'train_runtime': 772.4438, 'train_samples_per_second': 5.433, 'train_steps_per_second': 1.359, 'total_flos': 2381880829722624.0, 'train_loss': 0.3486320872533889, 'epoch': 3.0}


In [8]:
# Do prediction on the test set
trainer.infer()

PredictionOutput(predictions=array([[[ 1.3079702 , -0.2904181 ,  0.2586384 , ..., -0.11352221,
          0.07646519, -0.24405244],
        [ 6.789712  , -1.0005671 , -0.37727392, ..., -0.15953152,
         -1.2844036 , -1.0367423 ],
        [ 6.633817  , -0.39296174, -0.3189742 , ...,  0.12375297,
         -1.3064742 , -1.7904514 ],
        ...,
        [-2.1680937 , -2.1756527 , -0.4168364 , ...,  7.0710583 ,
         -2.2435071 ,  1.2722484 ],
        [-2.3051016 , -2.0976422 , -0.52687037, ...,  6.9871902 ,
         -2.1772678 ,  1.0500771 ],
        [-2.2385902 , -2.1926658 , -0.6458242 , ...,  7.004057  ,
         -2.118276  ,  1.0429472 ]],

       [[ 0.48977107, -0.26755586,  0.5443686 , ..., -0.7454087 ,
          0.7943655 ,  0.21615645],
        [ 6.5489154 , -0.512016  ,  0.19189848, ..., -1.5005919 ,
         -0.8510053 , -0.59021753],
        [ 6.7255936 , -0.09630061,  0.02421203, ..., -1.5355651 ,
         -0.6872942 , -0.8555626 ],
        ...,
        [-0.49812365, -1.